In [24]:
from langchain_core.tools import tool
from langchain.chat_models import init_chat_model
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent

In [27]:
@tool
def search_companies_by_industry(industry: str) -> str:
    """输入行业名称，返回一个公司列表ID。必须先用此工具获取 ID，才能用 search_people_by_company_list 查询人员。"""
    return "company_list_123"

@tool
def get_company_list_id_from_names(companies: list) -> str:
    """输入公司名称列表，返回一个公司列表ID，用于后续批量查询人员。"""
    return "company_list_from_names_456"
    
@tool
def expand_job_title(base_title: str) -> list:
    """
    使用大模型，根据一个职位名称生成相关职位名称列表。
    例如: "软件工程师" → ["软件工程师", "后端开发", "全栈工程师", "研发工程师"]
    """
    llm = init_chat_model("gpt-4o-mini", temperature=0)
    prompt = f"""
    你是一个人力资源专家。给定职位名称 "{base_title}"，
    请生成一个包含 3~5 个相关职位名称的 JSON 数组。
    只输出 JSON 数组，不要多余文字，不要用 markdown。
    """
    resp = llm.invoke(prompt)
    try:
        print('expand_job_title:', resp.content)
        import json
        job_list = json.loads(resp.content)
        if isinstance(job_list, list):
            return job_list
    except Exception:
        raise
    return [base_title]  # fallback

@tool
def search_people_by_company_list(company_list_id: str, job_titles: list) -> list:
    """
    输入公司列表ID和多个职位关键字，批量查找这些公司的人。
    job_titles 示例: ["软件工程师", "后端开发", "研发工程师"]
    """
    results = []
    for title in job_titles:
        results.append({
            "name": f"Test-{title}",
            "title": title,
            "company": "DemoCorp"
        })
    return results    

@tool
def get_phone_number_by_name(name: str) -> str:
    """输入一个人的姓名，返回手机号。"""
    phone_book = {
        "Test-软件工程师": "13800000001",
        "Test-后端开发": "13800000002",
        "Test-全栈工程师": "13800000003",
        "Test-研发工程师": "13800000004"
    }
    return phone_book.get(name, "未知")    

In [28]:
# ====== 初始化模型 ======
model = init_chat_model("gpt-4o-mini", temperature=0)

# ====== 内存保存器 ======
memory = MemorySaver()

# ====== 创建 ReAct Agent ======
agent = create_react_agent(
    model=model,
    tools=[
        search_companies_by_industry,
        get_company_list_id_from_names,
        expand_job_title,
        search_people_by_company_list,
        get_phone_number_by_name
    ],
    checkpointer=memory
)

In [30]:
def ask(user_query):
    config = {"configurable": {"thread_id": "demo-thread-3"}}
    events = agent.stream({"messages": [("user", user_query)]}, config)
    
    for event in events:
        if "agent" in event:
            for msg in event["agent"]["messages"]:
                msg.pretty_print()  # Agent 中间推理
    
        if "tools" in event:
            for msg in event["tools"]["messages"]:
                msg.pretty_print()  # 工具输入输出


In [31]:
ask("帮我查医疗行业公司的软件工程师")

================================== Ai Message ==================================
Tool Calls:
  search_companies_by_industry (call_yJSqBzmfGVKuON4EG3U2zUkl)
 Call ID: call_yJSqBzmfGVKuON4EG3U2zUkl
  Args:
    industry: 医疗
================================= Tool Message =================================
Name: search_companies_by_industry

company_list_123
================================== Ai Message ==================================
Tool Calls:
  expand_job_title (call_AaG3beNt60vQawqBx1S11emF)
 Call ID: call_AaG3beNt60vQawqBx1S11emF
  Args:
    base_title: 软件工程师
expand_job_title: [
    "前端开发工程师",
    "后端开发工程师",
    "全栈工程师",
    "软件开发工程师",
    "系统架构师"
]
================================= Tool Message =================================
Name: expand_job_title

["前端开发工程师", "后端开发工程师", "全栈工程师", "软件开发工程师", "系统架构师"]
================================== Ai Message ==================================
Tool Calls:
  search_people_by_company_list (call_KCydHJ2vHCfjv3KE6QnRXDYx)
 Call ID: call_KCydHJ2vHC

In [32]:
ask('帮我查这些公司（阿里巴巴, 腾讯）的软件工程师并给出手机号')

================================== Ai Message ==================================
Tool Calls:
  get_company_list_id_from_names (call_X2At6jLUTUe0jjn3NysY7X4o)
 Call ID: call_X2At6jLUTUe0jjn3NysY7X4o
  Args:
    companies: ['阿里巴巴', '腾讯']
================================= Tool Message =================================
Name: get_company_list_id_from_names

company_list_from_names_456
================================== Ai Message ==================================
Tool Calls:
  expand_job_title (call_RORfCBYdGjeP9g3JKnQNJcMx)
 Call ID: call_RORfCBYdGjeP9g3JKnQNJcMx
  Args:
    base_title: 软件工程师
expand_job_title: [
    "前端开发工程师",
    "后端开发工程师",
    "全栈工程师",
    "软件开发工程师",
    "系统架构师"
]
================================= Tool Message =================================
Name: expand_job_title

["前端开发工程师", "后端开发工程师", "全栈工程师", "软件开发工程师", "系统架构师"]
================================== Ai Message ==================================
Tool Calls:
  search_people_by_company_list (call_RnBk7DAlMH6tj5JaBkZppTz